# Basic Chatbot

##### Boilerplate code

In [3]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

##### Manually trimming history - just to know

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ChatMessageHistory

prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

conversation = {}

# {sessionid: [], sessionid: []}

def chat_with_bot(payload):
    input = payload['input']
    session_id = payload['session_id']

    if session_id not in conversation:
        conversation[session_id] = ChatMessageHistory()
    
    history = conversation[session_id]

    history.messages = history.messages[-2:]
    
    history.add_user_message(input)
    res = chain.invoke({"input": input, "history": history.messages})
    history.add_ai_message(res)
    print(f"history messages length: {len(history.messages)}")
    
    return res


res = chat_with_bot({"input": "My name is Praveen", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "What is google?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "Where is it located?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "What is my name?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

##### RunnableWithMessageHistory

In [ ]:
from collections import deque
from langchain_core.messages import BaseMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory


prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

store = {}

class WindowedChatMessageHistory(BaseChatMessageHistory):
    def __init__(self, k: int = 2):
        self._messages = deque(maxlen=k*2)

    @property
    def messages(self):
        return list(self._messages)

    def add_message(self, message: BaseMessage) -> None:
        self._messages.append(message)

    def clear(self):
        self._messages.clear()


def get_session_history(session_id):
    if session_id not in store:
         store[session_id] = WindowedChatMessageHistory(k=2)
    return store[session_id]


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

print(store)
res = chain_with_history.invoke(
    {"input": "My name is Praveen."},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


print(store)
res = chain_with_history.invoke(
    {"input": "What is google?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")

print(store)
res = chain_with_history.invoke(
    {"input": "Where is it located?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


print(store)
res = chain_with_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


{}
Okay, Praveen. 

{'user1': <__main__.WindowedChatMessageHistory object at 0x119ecad90>}
Google is a search engine. 

{'user1': <__main__.WindowedChatMessageHistory object at 0x119ecad90>}
Headquartered in Mountain View, California. 

{'user1': <__main__.WindowedChatMessageHistory object at 0x119ecad90>}
I do not know your name. 

